In [92]:
import pandas as pd
import re
import os
import openai

In [93]:
def load_excels(folder_path):
    excel_dfs = {}

    for file in os.listdir(folder_path):
        if file.endswith((".xlsx")):
            file_path = os.path.join(folder_path, file)

            cleaned_name = re.sub(r"[^a-zA-Z\s]", "", file)
            cleaned_name = "_".join(cleaned_name.split())
            df = pd.read_excel(file_path)
            excel_dfs[cleaned_name] = df
    return excel_dfs

In [94]:
folder_path = r"C:\Jupyter Notebook\Action Plans"
excel_dfs = load_excels(folder_path)

In [95]:
print(excel_dfs)

{'A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_xlsx':      Ref.                                             Action  \
0     1.1  As an integral part of the ‘healthy\nschool’ c...   
1     1.2  Develop and implement an integrated\nand holis...   
2     1.3  Develop and implement training\nprogrammes, in...   
3     1.4  Provide potable water in all learning\ncentres...   
4     1.5  Expand parenting programmes that\nincorporate ...   
..    ...                                                ...   
57   10.7  Enhance the research capabilities, training\na...   
58   10.8  Develop an obesity ‘knowledge translation’\npr...   
59   10.9  Align targets with outcome indicators in\nthe ...   
60  10.10  Develop an annual bulletin or score card on pr...   
61  10.11  Conduct a mid-term review of the Obesity Polic...   

                                     Lead Responsible  \
0                                                 DES   
1                                               

In [96]:
def create_dataframes(df_dict):
    for name, df in df_dict.items():
        globals()[name] = df

In [97]:
create_dataframes(excel_dfs)

In [98]:
df_names = []
for key, value in excel_dfs.items():
    df_names.append(key)

df_names

['A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_xlsx',
 'A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_xlsx',
 'Action_Plan_for_Rural_Developmentxlsx',
 'Climate_Action_Plan_xlsx',
 'Climate_Change_Action_Plan_xlsx',
 'Combinedxlsx',
 'Irelands_th_National_Biodiversity_Plan_to_xlsx',
 'Irelands_National_Action_Plan_on_Antimicrobial_Resistance_xlsx',
 'Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_xlsx',
 'National_Biodiversity_Action_Plan_xlsx',
 'Realising_our_Rural_Potential_Action_Plan_for_Rural_Developmentxlsx']

In [99]:
topics = ['carbon', 'neutrality', 'emissions', 'abatement', 
          'water', 'waterways', 'rivers', 'nitrates', 'eutrophication', 'effluent', 
          'discharge', 'inherit', 'nitrogen efficient', 'nitrogen fixing', 
          'succession', 'rural development', 'community', 'social sustainability', 'society', 'economic viability']

In [100]:

def identify_topics(df,text_col,topics):

    system_prompt = (
        "You are an assistant trained to analyze text and identify specific topics mentioned in the text. "
        "Your task is to carefully examine the provided text and identify significant mentions of the topics. "
        "Only consider a topic to be 'mentioned' if it is central to the discussion or if the text provides"
        "some meaningful information or opinion about the topic."
    )
    
    def extract_topics_from_text(text):

        user_prompt = (
            f"The topics to identify are:\n{topics}\n\n"
            "Below is a action plan. Identify which topics are significantly mentioned. "
            "Provide a concise list of only the topics that are clearly relevant "
            "If none are relevant, return an empty list.\n\n"
            f"Statement: {text}"
        )
        
        try:
            response = openai.chat.completions.create(
                model="gpt-4o-mini", 
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=400,  
                temperature=0.2
            )
            topics_mentioned = response.choices[0].message.content.strip()
            return eval(topics_mentioned) if topics_mentioned.strip().startswith("[") else []
        except Exception as e:
            print(f"Error processing text: {e}")
            return []

    df["Topics"] = df[text_col].apply(extract_topics_from_text)
    return df

In [101]:
A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic = identify_topics(A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_xlsx, "Action", topics)
A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic = A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic[["Action", "Lead Responsible", "Partners", "Timeframe", "Topics"]]
A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic = A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic[A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic["Topics"].astype(str) != "[]"]
A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic["Action"] = A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic["Action"].str.replace('\n',' ',regex = True)
A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic["Lead Responsible"] = A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic["Lead Responsible"].str.replace('\n',' ',regex = True)
A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic["Partners"] = A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic["Partners"].str.replace('\n',' ',regex = True)
A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic

,Action,Lead Responsible,Partners,Timeframe,Topics
3,Provide potable water in all learning centres ...,"DES, pre-schools, schools, colleges, universit...",NaN,2016–2025,"[water, potable water, community]"


In [104]:
A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_topic = identify_topics(A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_xlsx, "Actions", topics)
A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_topic = A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_topic[["Actions","Topics"]]
A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_topic = A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_topic[A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_topic["Topics"].astype(str) != "[]"]
A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_topic["Action"] = A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_topic["Actions"].str.replace('\n',' ',regex = True)
A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_topic.drop(columns = ["Actions"], inplace = True)
A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_topic

,Topics,Action


In [112]:
Action_Plan_for_Rural_Development_topic = identify_topics(Action_Plan_for_Rural_Developmentxlsx, "Action", topics)

In [113]:
Action_Plan_for_Rural_Development_topic = Action_Plan_for_Rural_Development_topic[["Action", "Responsible Bodies", "Timeline", "Topics"]]
Action_Plan_for_Rural_Development_topic = Action_Plan_for_Rural_Development_topic[Action_Plan_for_Rural_Development_topic["Topics"].astype(str) != "[]"]
Action_Plan_for_Rural_Development_topic["Action"] = Action_Plan_for_Rural_Development_topic["Action"].str.replace('\n',' ',regex = True)
Action_Plan_for_Rural_Development_topic["Lead Responsible"] = Action_Plan_for_Rural_Development_topic["Responsible Bodies"].str.replace('\n',' ',regex = True)
Action_Plan_for_Rural_Development_topic["Timeframe"] = Action_Plan_for_Rural_Development_topic["Timeline"].str.replace('\n',' ',regex = True)
Action_Plan_for_Rural_Development_topic.columns = Action_Plan_for_Rural_Development_topic.columns.str.strip()
Action_Plan_for_Rural_Development_topic

,Action,Responsible Bodies,Timeline,Topics,Lead Responsible,Timeframe
0,Support over 600 towns and villages through an...,"DAHRRGA, LAs",2017-2019,"[rural development, community, economic viabil...","DAHRRGA, LAs",2017-2019
13,Implement the GLAS Traditional Farm buildings ...,"Heritage Council,\nDAFM",2017-2019,"[rural development, community]","Heritage Council, DAFM",2017-2019
21,Develop a strategy for the growth and developm...,DoF,Ongoing,"[rural development, community, economic viabil...",DoF,Ongoing
28,"Plan and resource the Primary Care workforce, ...",HSE,Ongoing,"[community, rural development, social sustaina...",HSE,Ongoing
38,Implement the Framework Policy on Local & Comm...,"DHPCLG, other\nrelevant Government\nDepartment...",Q2 2017 to\nend 2019,"[rural development, community, social sustaina...","DHPCLG, other relevant Government Departments,...",Q2 2017 to end 2019
39,Engage with local communities on water quality...,"DHPCLG, LAs",Ongoing,"[water, waterways, community]","DHPCLG, LAs",Ongoing
40,Through the National Dialogue for Climate Chan...,DCCAE,Q1 2017,"[carbon, rural development, community, social ...",DCCAE,Q1 2017
43,Develop a new and effective rural proofing mod...,"DAHRRGA,\nDepartment of the\nTaoiseach",Q4 2017,"[rural development, community, society]","DAHRRGA, Department of the Taoiseach",Q4 2017
46,Deliver the LEADER measure of the Rural Develo...,"DAHRRGA, Leader\nLocal Action Groups,\nLAs, LD...",2017-2020,"[rural development, community, social sustaina...","DAHRRGA, Leader Local Action Groups, LAs, LDCs...",2017-2020
47,Invest €12m in 7 Fisheries Local Action Groups...,DAFM,2017-2020,"[community, social sustainability, economic vi...",DAFM,2017-2020


In [114]:
Climate_Action_Plan_topic = identify_topics(Climate_Action_Plan_xlsx, "Action", topics)

In [115]:
Climate_Action_Plan_topic = Climate_Action_Plan_topic[["Action","Topics"]]
Climate_Action_Plan_topic = Climate_Action_Plan_topic[Climate_Action_Plan_topic["Topics"].astype(str) != "[]"]
Climate_Action_Plan_topic["Action"] = Climate_Action_Plan_topic["Action"].str.replace('\n',' ',regex = True)
Climate_Action_Plan_topic

,Action,Topics
1,Conduct a review of greenhouse gas emissions o...,"[emissions, neutrality]"
2,Streamline the reporting process to distinguis...,"[emissions, abatement, neutrality]"
5,Complete the statutory process for the legal a...,"[carbon, neutrality, emissions]"
6,Prepare Climate Action Plan 2022 to fully refl...,[carbon]
13,Promote the implementation of the study “Skill...,"[carbon, neutrality, emissions]"
...,...,...
422,"Publish the Seafood Carbon Footprint Study, pr...",[carbon]
423,Complete a pilot study on improving the carbon...,[carbon]
425,Develop a study to test different seaweeds cul...,[emissions]
438,Develop a policy statement on mineral explorat...,"[carbon, neutrality, economic viability]"


In [116]:
Climate_Change_Action_Plan_topic = identify_topics(Climate_Change_Action_Plan_xlsx, "Action", topics)

In [117]:
Climate_Change_Action_Plan_topic = Climate_Change_Action_Plan_topic[["Action","Topics"]]
Climate_Change_Action_Plan_topic = Climate_Change_Action_Plan_topic[Climate_Change_Action_Plan_topic["Topics"].astype(str) != "[]"]
Climate_Change_Action_Plan_topic["Action"] = Climate_Change_Action_Plan_topic["Action"].str.replace('\n',' ',regex = True)
Climate_Change_Action_Plan_topic

,Action,Topics
0,Evaluate in detail the changes required to ado...,"[carbon, neutrality, emissions]"
2,Update Cabinet procedures to ensure that all G...,"[carbon, neutrality, emissions]"
4,Commence the process of forming carbon budgets...,"[carbon, neutrality, emissions]"
5,Require budgetary rules to show the Exchequer ...,[emissions]
7,Implement a carbon tax rate of at least €80 pe...,"[carbon, neutrality, emissions]"
8,Reform the Public Spending Code to increase th...,"[carbon, neutrality, emissions]"
12,"Ensure that, within their mandates, the select...","[carbon, neutrality, emissions]"
13,Strengthen our delivery of public funding for ...,"[carbon, neutrality, emissions, economic viabi..."
27,Design and implement the RESS. Increase the vo...,"[community, economic viability]"
33,Engage with the cement and food and drink indu...,[abatement]


In [118]:
Irelands_th_National_Biodiversity_Plan_to_topic = identify_topics(Irelands_th_National_Biodiversity_Plan_to_xlsx, "Action", topics)

In [119]:
Irelands_th_National_Biodiversity_Plan_to_topic = Irelands_th_National_Biodiversity_Plan_to_topic[["Target", "Action", "Partner(s)", "Topics"]]
Irelands_th_National_Biodiversity_Plan_to_topic = Irelands_th_National_Biodiversity_Plan_to_topic[Irelands_th_National_Biodiversity_Plan_to_topic["Topics"].astype(str) != "[]"]
Irelands_th_National_Biodiversity_Plan_to_topic["Action"] = Irelands_th_National_Biodiversity_Plan_to_topic["Action"].str.replace('\n',' ',regex = True)
Irelands_th_National_Biodiversity_Plan_to_topic["Partners"] = Irelands_th_National_Biodiversity_Plan_to_topic["Partner(s)"].str.replace('\n',' ',regex = True)
Irelands_th_National_Biodiversity_Plan_to_topic["Target"] = Irelands_th_National_Biodiversity_Plan_to_topic["Target"].str.replace('\n',' ',regex = True)
Irelands_th_National_Biodiversity_Plan_to_topic

,Target,Action,Partner(s),Topics,Partners
29,"By 2024, communities are working together to a...","NPWS, DAFM, the Department of Rural and Commun...","NPWS,\nDAFM, DRCD\nand DECC","[community, rural development]","NPWS, DAFM, DRCD and DECC"
37,"By 2026, Ireland has mainstreamed the engageme...","DAFM, DHLGH and other relevant stakeholders wi...","DAFM,\nDHLGH","[community, social sustainability]","DAFM, DHLGH"
55,"By 2027, optimised opportunities under agricul...",DAFM will ensure that farmers are incentivised...,DAFM,"[rural development, community, society]",DAFM
70,"By 2024, ensure that the impact of water quali...","DAFM, DHLGH and NPWS will support the undertak...","DAFM,\nDHLGH and\nNPWS","[nitrates, rural development, community]","DAFM, DHLGH and NPWS"
76,"By 2026, Bord na Móna to rehabilitate 33,000 h...",Bord na Móna as operator of the Enhanced Decom...,"Bord na\nMóna, NPWS\nand relevant\nstakeholders","[carbon, emissions, water]","Bord na Móna, NPWS and relevant stakeholders"
80,"By 2025, nature-based solutions are contributi...",To support the National Climate Objective of a...,All relevant\nstakeholders,"[carbon, neutrality, environmentally sustainab...",All relevant stakeholders
81,"By 2025, nature-based solutions are contributi...",To support the National Climate Objective of a...,NPWS,"[carbon, neutrality, water, community, rural d...",NPWS
82,"By 2027, protection and restoration measures d...","Relevant bodies such as DHLGH, DAFM, Local Aut...",All Relevant\nBodies,"[water, waterways, community]",All Relevant Bodies
83,"By 2027, protection and restoration measures d...",Uisce Éireann will implement its Water Service...,Uisce Éireann,"[water, waterways, community, social sustainab...",Uisce Éireann
84,"By 2027, protection and restoration measures d...",DHLGH and LAWPRO will ensure that high status ...,DHLGH and\nLAWPRO,"[water, waterways]",DHLGH and LAWPRO


In [120]:
Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic = identify_topics(Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic, "Activities", topics)

In [125]:
Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic = Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic[["Strategic interventions", "Activities", "Responsible", "Topics"]]
Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic = Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic[Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic["Topics"].astype(str) != "[]"]
Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic["Strategic interventions"] = Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic["Strategic interventions"].str.replace('\n',' ',regex = True)
Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic["Action"] = Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic["Activities"].str.replace('\n',' ',regex = True)
Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic["Lead Responsible"] = Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic["Responsible"].str.replace('\n',' ',regex = True)
Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic

,Strategic interventions,Activities,Responsible,Topics,Action,Lead Responsible
28,B. Minimise the risk of spread of AMR by water...,B.1. Examine mechanisms and approaches to redu...,NaN,"[discharge, effluent]",B.1. Examine mechanisms and approaches to redu...,NaN
39,3.8 Minimise the risk of spread of AMR by wate...,3.8.1 Assess the extent to which invtervention...,EPA,"[discharge, effluent]",3.8.1 Assess the extent to which invtervention...,EPA


In [123]:
Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic = identify_topics(Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_xlsx, "Strategic Interventions/Actions", topics)

In [127]:
Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic = Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic[["Activity","Strategic Interventions/Actions", "Timeframe", "Responsibility", "Topics"]]
Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic = Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic[Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic["Topics"].astype(str) != "[]"]
Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic["Strategic Interventions"] = Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic["Strategic Interventions/Actions"].str.replace('\n',' ',regex = True)
Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic["Action"] = Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic["Activity"].str.replace('\n',' ',regex = True)
Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic["Lead Responsible"] = Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic["Responsibility"].str.replace('\n',' ',regex = True)
Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic

,Activity,Strategic Interventions/Actions,Timeframe,Responsibility,Topics,Strategic Interventions,Action,Lead Responsible
71,NaN,Develop a scoping document investigating poten...,2024,"EPA, DHPLG,\nIrish Water,\nHSE, DAFM","[water, effluent, discharge]",Develop a scoping document investigating poten...,NaN,"EPA, DHPLG, Irish Water, HSE, DAFM"
78,NaN,Enhance collaboration and promote community of...,2021,"HSE, HIQA,\nPrivate Providers","[community, social sustainability, society]",Enhance collaboration and promote community of...,NaN,"HSE, HIQA, Private Providers"


In [128]:
National_Biodiversity_Action_Plan_topic = identify_topics(National_Biodiversity_Action_Plan_xlsx, "Action", topics)

In [129]:
National_Biodiversity_Action_Plan_topic = National_Biodiversity_Action_Plan_topic[["Action","Lead/key partners", "Timeframe", "Performance indicators", "Topics"]]
National_Biodiversity_Action_Plan_topic = National_Biodiversity_Action_Plan_topic[National_Biodiversity_Action_Plan_topic["Topics"].astype(str) != "[]"]
National_Biodiversity_Action_Plan_topic["Action"] = National_Biodiversity_Action_Plan_topic["Action"].str.replace('\n',' ',regex = True)
National_Biodiversity_Action_Plan_topic["Lead Responsible"] = National_Biodiversity_Action_Plan_topic["Lead/key partners"].str.replace('\n',' ',regex = True)
National_Biodiversity_Action_Plan_topic["Performance indicators"] = National_Biodiversity_Action_Plan_topic["Performance indicators"].str.replace('\n',' ',regex = True)
National_Biodiversity_Action_Plan_topic

,Action,Lead/key partners,Timeframe,Performance indicators,Topics,Lead Responsible
15,1.1.16. Establish and implement mechanisms for...,relevant\ngovernment\ndepartments,2020,1. Number of mechanisms established for paymen...,"[carbon, biodiversity, economic viability]",relevant government departments
21,1.2.4. Ensure the various provisions of the Fo...,dAFM,2018,1. Commencement orders for all provisions of t...,"[water, biodiversity, environment]",dAFM
29,2.1.5. Support research on economic and societ...,"EPA, dchg, DAFM,\nIFNC, NESC,\nCSO, Academia\n...",2017-2021,1. Number of relevant research projects initiated,"[society, economic viability]","EPA, dchg, DAFM, IFNC, NESC, CSO, Academia and..."
34,2.1.10. Continue forest research programme on ...,"DAFM, COFORD,\nForest Service,\nAcademia\nand ...",2017-2021,1. Number of published research papers and rep...,"[carbon, water, emissions]","DAFM, COFORD, Forest Service, Academia and Res..."
49,3.1.1. Engage with local communities and stake...,Relevant\ngovernment\ndepartments and\nagencie...,2017-2021,1. Consultation with community groups undertaken,[community],"Relevant government departments and agencies, ..."
57,4.1.1. Development and implementation of agri-...,"dAFM, Teagasc",2017-2021,1. Length/area of habitats under AES measures ...,"[rural development, community, social sustaina...","dAFM, Teagasc"
66,"4.2.1. Continue to protect, enhance and monito...","EPA, dAFM\nDHPLG, IFI, DCHG,\nLocal Authoritie...",2017-2021,1. The status of water quality in Ireland 2. N...,"[water, waterways, effluent, discharge]","EPA, dAFM DHPLG, IFI, DCHG, Local Authorities,..."
68,4.2.3. Implement recommendations of EPA STRIVE...,"EPA, Local\nAuthorities, DAFM",2017-2021,1. High status catchment delineation and prior...,"[water, waterways, community, social sustainab...","EPA, Local Authorities, DAFM"
85,5.1.2. Implement measures to achieve good ecol...,"dhPlg, dccAE,\nMarine Institute,\ndAFM,\nIrish...",2017-2021,1. Progress on measures under MSFD 2. River Ba...,"[water, waterways, community, social sustainab...","dhPlg, dccAE, Marine Institute, dAFM, Irish Wa..."


In [130]:
Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic = identify_topics(Realising_our_Rural_Potential_Action_Plan_for_Rural_Developmentxlsx, "Action", topics)

In [131]:
Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic = Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic[["Action", "Responsible Bodies", "Timeline", "Topics"]]
Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic = Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic[Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic["Topics"].astype(str) != "[]"]
Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic["Action"] = Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic["Action"].str.replace('\n',' ',regex = True)
Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic["Lead Responsible"] = Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic["Responsible Bodies"].str.replace('\n',' ',regex = True)
Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic["Timeframe"] = Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic["Timeline"].str.replace('\n',' ',regex = True)
Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic

,Action,Responsible Bodies,Timeline,Topics,Lead Responsible,Timeframe
0,Support over 600 towns and villages through an...,"DAHRRGA, LAs",2017-2019,"[rural development, community, economic viabil...","DAHRRGA, LAs",2017-2019
13,Implement the GLAS Traditional Farm buildings ...,"Heritage Council,\nDAFM",2017-2019,"[rural development, community]","Heritage Council, DAFM",2017-2019
18,Provide funding for Volunteer Centres and Volu...,DHPCLG,Ongoing,"[community, rural development]",DHPCLG,Ongoing
21,Develop a strategy for the growth and developm...,DoF,Ongoing,"[rural development, community, economic viabil...",DoF,Ongoing
28,"Plan and resource the Primary Care workforce, ...",HSE,Ongoing,"[community, rural development, social sustaina...",HSE,Ongoing
36,Implement the Framework Policy on Local & Comm...,"DHPCLG, other\nrelevant Government\nDepartment...",Q2 2017 to\nend 2019,"[rural development, community, social sustaina...","DHPCLG, other relevant Government Departments,...",Q2 2017 to end 2019
37,Engage with local communities on water quality...,"DHPCLG, LAs",Ongoing,"[water, waterways, community]","DHPCLG, LAs",Ongoing
38,Through the National Dialogue for Climate Chan...,DCCAE,Q1 2017,"[carbon, rural development, community, social ...",DCCAE,Q1 2017
41,Develop a new and effective rural proofing mod...,"DAHRRGA,\nDepartment of the\nTaoiseach",Q4 2017,"[rural development, community, society]","DAHRRGA, Department of the Taoiseach",Q4 2017
44,Deliver the LEADER measure of the Rural Develo...,"DAHRRGA, Leader\nLocal Action Groups,\nLAs, LD...",2017-2020,"[rural development, community, social sustaina...","DAHRRGA, Leader Local Action Groups, LAs, LDCs...",2017-2020


In [138]:
A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic['File Name'] = "A Healthy Weight for Ireland - Obesity Policy and Action Plan 2016-2025"
A_Waste_Action_Plan_for_a_Circular_Economy_Irelands_National_Waste_Policy_topic['File Name'] = "A Waste Action Plan for a Circular Economy Ireland's National Waste Policy 2020-2025"
Action_Plan_for_Rural_Development_topic['File Name'] = "Action Plan for Rural Development"
Climate_Action_Plan_topic['File Name'] = "Climate Action Plan 2021"
Climate_Change_Action_Plan_topic['File Name'] = "Climate Change Action Plan 2019"
Irelands_th_National_Biodiversity_Plan_to_topic['File Name'] = "Ireland's 4th National Biodiversity Plan 2023 to 2030"
Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic['File Name'] = "Ireland’s National Action Plan on Antimicrobial Resistance 2017 – 2020"
Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic['File Name'] = "Ireland’s Second One Health National Action Plan on Antimicrobial Resistance 2021 - 2025"
National_Biodiversity_Action_Plan_topic['File Name'] = "National Biodiversity Action Plan 2017 - 2021"
Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic['File Name'] = "Realising our Rural Potential - Action Plan for Rural Development"

In [139]:
dataframes = [  A_Healthy_Weight_for_Ireland_Obesity_Policy_and_Action_Plan_topic,
                Action_Plan_for_Rural_Development_topic,
                Climate_Action_Plan_topic,
                Climate_Change_Action_Plan_topic,
                Irelands_th_National_Biodiversity_Plan_to_topic,
                Irelands_National_Action_Plan_on_Antimicrobial_Resistance_topic,
                Irelands_Second_One_Health_National_Action_Plan_on_Antimicrobial_Resistance_topic,
                National_Biodiversity_Action_Plan_topic,
                Realising_our_Rural_Potential_Action_Plan_for_Rural_Development_topic
             ]

required_columns = [
    "Strategic interventions", "Action", "Lead Responsible", 
    "Partners", "Timeframe", "Topics", "File Name", 
    "Target", "Performance indicators"
]

processed_df = []

for df in dataframes:
    df_selected = df.reindex(columns = required_columns)
    processed_df.append(df_selected)

final_df = pd.concat(processed_df, ignore_index = True)

print(final_df)

    Strategic interventions  \
0                       NaN   
1                       NaN   
2                       NaN   
3                       NaN   
4                       NaN   
..                      ...   
256                     NaN   
257                     NaN   
258                     NaN   
259                     NaN   
260                     NaN   

                                                Action  \
0    Provide potable water in all learning centres ...   
1    Support over 600 towns and villages through an...   
2    Implement the GLAS Traditional Farm buildings ...   
3    Develop a strategy for the growth and developm...   
4    Plan and resource the Primary Care workforce, ...   
..                                                 ...   
256  Progress the major roads projects detailed in ...   
257  Conduct a full review of public transport poli...   
258  Work with rural communities to assess and impl...   
259  Introduce a trial to lower the water level

In [141]:
final_df.head()

,Strategic interventions,Action,Lead Responsible,Partners,Timeframe,Topics,File Name,Target,Performance indicators
0,NaN,Provide potable water in all learning centres ...,"DES, pre-schools, schools, colleges, universit...",NaN,2016–2025,"[water, potable water, community]",A Healthy Weight for Ireland - Obesity Policy ...,NaN,NaN
1,NaN,Support over 600 towns and villages through an...,"DAHRRGA, LAs",NaN,2017-2019,"[rural development, community, economic viabil...",Action Plan for Rural Development,NaN,NaN
2,NaN,Implement the GLAS Traditional Farm buildings ...,"Heritage Council, DAFM",NaN,2017-2019,"[rural development, community]",Action Plan for Rural Development,NaN,NaN
3,NaN,Develop a strategy for the growth and developm...,DoF,NaN,Ongoing,"[rural development, community, economic viabil...",Action Plan for Rural Development,NaN,NaN
4,NaN,"Plan and resource the Primary Care workforce, ...",HSE,NaN,Ongoing,"[community, rural development, social sustaina...",Action Plan for Rural Development,NaN,NaN


In [142]:
final_df.to_excel(r"Combined.xlsx")